一.复现NIN网络

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021PyTorchDL/WEEK9/58.png?versionId=CAEQFRiBgIC0zIWfxxciIGE0ZjkyMzI1ZTE3MzQ3MmQ5NDhiZWFlN2UyMmFiYTQ0)

In [14]:
import torch
from torch import nn
from torch.nn import functional as F
from torchinfo import summary
from torch_receptive_field import receptive_field

In [15]:
data=torch.ones(size=(10,3,32,32))
class NiN(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = nn.Sequential(nn.Conv2d(3,192,5,padding=2),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,160,1),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(160,96,1),nn.ReLU(inplace=True)
                                    ,nn.MaxPool2d(kernel_size=3,stride=2)
                                    ,nn.Dropout(0.25))
        self.block2 = nn.Sequential(nn.Conv2d(96,192,5,padding=2),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,192,1),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,192,1),nn.ReLU(inplace=True)
                                    ,nn.MaxPool2d(kernel_size=3,stride=2)
                                    ,nn.Dropout(0.25))
        self.block3 = nn.Sequential(nn.Conv2d(192,192,3,padding=1),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,192,1),nn.ReLU(inplace=True)
                                    ,nn.Conv2d(192,10,1),nn.ReLU(inplace=True)
                                    ,nn.AvgPool2d(7,stride=1)
                                    ,nn.Softmax(dim=1))
    def forward(self,x):
        output = self.block3(self.block2(self.block1(x)))
        return output

In [16]:
net=NiN()
net(data).shape


torch.Size([10, 10, 1, 1])

In [17]:
summary(net,input_size=(10,3,32,32),device="cpu")

Layer (type:depth-idx)                   Output Shape              Param #
NiN                                      [10, 10, 1, 1]            --
├─Sequential: 1-1                        [10, 96, 15, 15]          --
│    └─Conv2d: 2-1                       [10, 192, 32, 32]         14,592
│    └─ReLU: 2-2                         [10, 192, 32, 32]         --
│    └─Conv2d: 2-3                       [10, 160, 32, 32]         30,880
│    └─ReLU: 2-4                         [10, 160, 32, 32]         --
│    └─Conv2d: 2-5                       [10, 96, 32, 32]          15,456
│    └─ReLU: 2-6                         [10, 96, 32, 32]          --
│    └─MaxPool2d: 2-7                    [10, 96, 15, 15]          --
│    └─Dropout: 2-8                      [10, 96, 15, 15]          --
├─Sequential: 1-2                        [10, 192, 7, 7]           --
│    └─Conv2d: 2-9                       [10, 192, 15, 15]         460,992
│    └─ReLU: 2-10                        [10, 192, 15, 15]         -